In [ ]:
using Pkg; 
Pkg.activate("."); 
# Pkg.instantiate()
using Symbolics;
# Pkg.add("Rotations")
using Rotations

  Activating project at `d:\github_proj\Flexia\theory`
   Resolving package versions...
    Updating `D:\github_proj\Flexia\theory\Project.toml`
  [6038ab10] + Rotations v1.7.1
    Updating `D:\github_proj\Flexia\theory\Manifest.toml`
  [94ee1d12] + Quaternions v0.7.6
  [c1ae055f] + RealDot v0.1.0
  [6038ab10] + Rotations v1.7.1
Precompiling project...
  ✓ Rotations → RotationsRecipesBaseExt
  1 dependency successfully precompiled in 2 seconds. 141 already precompiled.


In [3]:
@variables xbi, ybi, θbi, xbj, ybj, θbj 
@variables xci, yci, θci, xcj, ycj, θcj

6-element Vector{Num}:
 xci
 yci
 θci
 xcj
 ycj
 θcj

# Fixed Joint

The constraint equations looks as the following:
$$\begin{equation}
    \mathbf{F}_{f}(\mathbf{x}) = \begin{cases}
        x_{b1} - x_{c1}\\
        y_{b1} - y_{c1}\\
        \theta_{b1} - \theta_{c1}\\
    \end{cases}
\end{equation}
$$
The derivatives (jacoby matrix) is as follow:
$$\begin{equation}
    \mathbf{J}_{f}(\mathbf{x}) = \begin{bmatrix}
        1 & 0 & 0\\
        0 & 1 & 0\\
        0 & 0 & 1\\
    \end{bmatrix}
\end{equation}
$$

# Hinge Joint
Given:

Hinge Joint connsects `body1` and `body2` at points $\mathbf{p}^{b1}_{c}$ and $\mathbf{p}^{b2}_{c}$ defined in body's local frame. 

The constraint equations looks 

$$
\mathbf{R}_{b1} \mathbf{p}^{b1}_{c} + \mathbf{p}_{b1} = \mathbf{R}_{b2} \mathbf{p}^{b2}_{c} + \mathbf{p}_{b2} 
$$

$$
    \mathbf{F}_{h}(\mathbf{x}) = \begin{cases}
        cos(\theta_{b1})*x^{b1}_{c} - sin(\theta_{b1})*y^{b1}_{c} + x_{b1} - cos(\theta_{b2})*x^{b2}_{c} + sin(\theta_{b2})*y^{b1}_c - x_{b2}\\
        sin(\theta_{b1})*x^{b1}_{c} + cos(\theta_{b1})*y^{b1}_{c} + y_{b1} - sin(\theta_{b2})*x^{b2}_{c} - cos(\theta_{b2})*y^{b1}_c - y_{b2}\\
    \end{cases}
$$


In [13]:
Ri = RotZ(θbi)[1:2, 1:2]
Rj = RotZ(θbj)[1:2, 1:2]
using LinearAlgebra
f = Ri * [xci, yci] + [xbi, ybi] - (Rj * [xcj, ycj] + [xbj, ybj])
dist = sum((f).^2)
Symbolics.gradient(dist, [xbi, ybi, θbi, xbj, ybj, θbj])

6-element Vector{Num}:
                                                                                                                                         2(xbi - xbj + xci*cos(θbi) - xcj*cos(θbj) - yci*sin(θbi) + ycj*sin(θbj))
                                                                                                                                         2(ybi - ybj + xci*sin(θbi) - xcj*sin(θbj) + yci*cos(θbi) - ycj*cos(θbj))
 2(-xci*sin(θbi) - yci*cos(θbi))*(xbi - xbj + xci*cos(θbi) - xcj*cos(θbj) - yci*sin(θbi) + ycj*sin(θbj)) + 2(ybi - ybj + xci*sin(θbi) - xcj*sin(θbj) + yci*cos(θbi) - ycj*cos(θbj))*(xci*cos(θbi) - yci*sin(θbi))
                                                                                                                                        -2(xbi - xbj + xci*cos(θbi) - xcj*cos(θbj) - yci*sin(θbi) + ycj*sin(θbj))
                                                                                                                                        -

In [17]:
Symbolics.jacobian(f, [xbi, ybi, θbi, xbj, ybj, θbj])'

6×2 adjoint(::Matrix{Num}) with eltype Num:
                            1                             0
                            0                             1
 -xci*sin(θbi) - yci*cos(θbi)   xci*cos(θbi) - yci*sin(θbi)
                           -1                             0
                            0                            -1
  xcj*sin(θbj) + ycj*cos(θbj)  -xcj*cos(θbj) + ycj*sin(θbj)

In [18]:
f

2-element Vector{Num}:
 xbi - xbj + xci*cos(θbi) - xcj*cos(θbj) - yci*sin(θbi) + ycj*sin(θbj)
 ybi - ybj + xci*sin(θbi) - xcj*sin(θbj) + yci*cos(θbi) - ycj*cos(θbj)

In [15]:
Symbolics.gradient(f[2], [xbi, ybi, θbi, xbj, ybj, θbj])

6-element Vector{Num}:
                            0
                            1
  xci*cos(θbi) - yci*sin(θbi)
                            0
                           -1
 -xcj*cos(θbj) + ycj*sin(θbj)